In [1]:
# this line is only used on AWS cluster
print("Welcome to my EMR Notebook!")

Welcome to my EMR Notebook!


In [2]:
# this line is only used on AWS cluster
%%info

UsageError: Cell magic `%%info` not found.


In [4]:
# this line is only used on AWS cluster
sc.list_packages()

NameError: name 'sc' is not defined

In [4]:
# those lines were only used on AWS cluster
sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 
sc.install_pypi_package("matplotlib") #Install matplotlib from given PyPI repository
sc.install_pypi_package("seaborn==0.10.1") #Install seaborn version 0.10.1
sc.install_pypi_package("ipython==7.17.0") #install ipython version 7.17.0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# this line is only used on AWS cluster
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'SparkContext' object has no attribute 'list_packages'
Traceback (most recent call last):
AttributeError: 'SparkContext' object has no attribute 'list_packages'



In [1]:
# import libraries
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession,Window
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression,GBTClassifier, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
from pyspark.ml.feature import StandardScaler, StringIndexer, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import re
import datetime

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt



In [2]:
# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

In [3]:
# session overview
spark.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1598254678553'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.port', '42563'),
 ('spark.app.name', 'Sparkify'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.host', 'd55fb14ecc64'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [4]:
spark

## Load and Clean Dataset
Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids.

In [5]:
# Read in full sparkify dataset
#user_log = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json" #whole data on cloud
#user_log = "s3://aws-emr-resources-057584263306-eu-central-1/notebooks/e-1DYCDZU9IGL1MSTRN75J0HLSI/mini_sparkify_event_data.json" #mini data on cloud
user_log = "mini_sparkify_event_data.json" # mini data local
df = spark.read.json(user_log)
#df.head()

In [6]:
# check columns
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [7]:
# show first 5 rows
df.show(n=5)

+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|          artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|  Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Boston-Cambridge-...|   PUT|NextSong|1538331630000| 

In [8]:
# count rows
df.count()

286500

In [9]:
### Get count of both null and missing values in pyspark
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId| song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
| 58392|   0|     8346|  8346|            0|    8346| 58392|    0|    8346|     0|   0|        8346|        0|58392|     0|  0|     8346|     0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+



In [10]:
# check "userId" with missing "firstName"
df.select(['userId']).where(df.firstName.isNull()).dropDuplicates().show()

+------+
|userId|
+------+
|      |
+------+



In [7]:
# show userId who's first name is missing
#df.where(df.userId == "1261737").show() # on whole data
df.where(df.userId == "").show(5) # on mini data

+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|artist|      auth|firstName|gender|itemInSession|lastName|length|level|location|method| page|registration|sessionId|song|status|           ts|userAgent|userId|
+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|  null|Logged Out|     null|  null|          100|    null|  null| free|    null|   GET| Home|        null|        8|null|   200|1538355745000|     null|      |
|  null|Logged Out|     null|  null|          101|    null|  null| free|    null|   GET| Help|        null|        8|null|   200|1538355807000|     null|      |
|  null|Logged Out|     null|  null|          102|    null|  null| free|    null|   GET| Home|        null|        8|null|   200|1538355841000|     null|      |
|  null|Logged Out|     null|  nul

In [8]:
# count rows with "userId" as 1261737
#df.where(df.userId == "1261737").count() # on whole data
df.where(df.userId == "").count() # on mini data

8346

In [7]:
# delete rows with "userId" as 1261737
#df_clean = df.filter(df.userId != "1261737") #on the whole data 
df_clean = df.filter(df.userId != "") #on the mini data 
df_clean.count()

278154

In [8]:
del df

In [10]:
# check unique userId 
df_clean.select('userId').dropDuplicates().count()

225

In [15]:
# show unique IDs
df_clean.select("userId").dropDuplicates().sort("userId").show(5) 

+------+
|userId|
+------+
|    10|
|   100|
|100001|
|100002|
|100003|
+------+
only showing top 5 rows



In [16]:
# check rows with missing value in "artist" 
df_clean.where(df_clean.artist.isNull()).show(5)

+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+---------------+-------------+---------+----+------+-------------+--------------------+------+
|artist|     auth|firstName|gender|itemInSession|lastName|length|level|            location|method|           page| registration|sessionId|song|status|           ts|           userAgent|userId|
+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+---------------+-------------+---------+----+------+-------------+--------------------+------+
|  null|Logged In|    Colin|     M|           54| Freeman|  null| paid|     Bakersfield, CA|   PUT|Add to Playlist|1538173362000|       29|null|   200|1538352905000|Mozilla/5.0 (Wind...|    30|
|  null|Logged In|    Micah|     M|           84|    Long|  null| free|Boston-Cambridge-...|   GET|    Roll Advert|1538331630000|        8|null|   200|1538353150000|"Mozilla/5.0 (Win...|     9|
|  null|Logged In|    Micah|  

In [11]:
# list activities of a user who has a missing value in the artist column 1009070
# df_clean.select(["artist","userId", "firstname", "page", "level", "song"]).where(df_clean.userId == "1009070").show() # on whole data
df_clean.select(["artist","userId", "firstname", "page", "level", "song"]).where(df_clean.userId == "30").show() # on mini data

+--------------------+------+---------+---------------+-----+--------------------+
|              artist|userId|firstname|           page|level|                song|
+--------------------+------+---------+---------------+-----+--------------------+
|      Martha Tilston|    30|    Colin|       NextSong| paid|           Rockpools|
|        Adam Lambert|    30|    Colin|       NextSong| paid|   Time For Miracles|
|           Daft Punk|    30|    Colin|       NextSong| paid|Harder Better Fas...|
|        Starflyer 59|    30|    Colin|       NextSong| paid|Passengers (Old A...|
|                null|    30|    Colin|Add to Playlist| paid|                null|
|            Frumpies|    30|    Colin|       NextSong| paid|          Fuck Kitty|
|Edward Sharpe & T...|    30|    Colin|       NextSong| paid|                Jade|
|         Stan Mosley|    30|    Colin|       NextSong| paid|   So-Called Friends|
|             Orishas|    30|    Colin|       NextSong| paid|           Represent|
|   

In [18]:
#check page related to "null" artist
df_clean.select('page').where(df_clean.artist.isNull()).groupBy('page').count().show()
# there is no "next song" page in the listed page categories, artist name is "null" for all the other pages, except "next song" page

+--------------------+-----+
|                page|count|
+--------------------+-----+
|              Cancel|   52|
|    Submit Downgrade|   63|
|         Thumbs Down| 2546|
|                Home|10082|
|           Downgrade| 2055|
|         Roll Advert| 3933|
|              Logout| 3226|
|       Save Settings|  310|
|Cancellation Conf...|   52|
|               About|  495|
|            Settings| 1514|
|     Add to Playlist| 6526|
|          Add Friend| 4277|
|           Thumbs Up|12551|
|                Help| 1454|
|             Upgrade|  499|
|               Error|  252|
|      Submit Upgrade|  159|
+--------------------+-----+



In [19]:
# check artist column
df_clean.select("artist").dropDuplicates().sort("artist").show(5) 

+--------------------+
|              artist|
+--------------------+
|                null|
|                 !!!|
|        & And Oceans|
|'N Sync/Phil Collins|
|        'Til Tuesday|
+--------------------+
only showing top 5 rows



### Digging in the missing values

In [7]:
# delete missing values in "artist" column
df_no_null = df_clean.filter(df_clean.artist != "")

### Get count of both null and missing values
df_no_null.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_no_null.columns]).show() 
# there is no missing value in any column after deleting missing "artist"
df_no_null.count()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
|     0|   0|        0|     0|            0|       0|     0|    0|       0|     0|   0|           0|        0|   0|     0|  0|        0|     0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+



228108

In [29]:
# check unique userId 
df_no_null.select('userId').dropDuplicates().count() # unique userId number did not change

225

In [22]:
# check tags in categorical columns
for item in ['auth','gender','level','method','page','sessionId','status','userAgent','location']:
    df_clean.select(item).dropDuplicates().show()

+---------+
|     auth|
+---------+
|Cancelled|
|Logged In|
+---------+

+------+
|gender|
+------+
|     F|
|     M|
+------+

+-----+
|level|
+-----+
| free|
| paid|
+-----+

+------+
|method|
+------+
|   PUT|
|   GET|
+------+

+--------------------+
|                page|
+--------------------+
|              Cancel|
|    Submit Downgrade|
|         Thumbs Down|
|                Home|
|           Downgrade|
|         Roll Advert|
|              Logout|
|       Save Settings|
|Cancellation Conf...|
|               About|
|            Settings|
|     Add to Playlist|
|          Add Friend|
|            NextSong|
|           Thumbs Up|
|                Help|
|             Upgrade|
|               Error|
|      Submit Upgrade|
+--------------------+

+---------+
|sessionId|
+---------+
|       29|
|       26|
|      474|
|      964|
|     1697|
|     1806|
|     2040|
|     1950|
|     2214|
|      418|
|       65|
|      541|
|      558|
|     1010|
|     1224|
|     1277|
|     1258

In [23]:
# check location column
df_clean.select("location").dropDuplicates().show(5) 
df_clean.select("location").dropDuplicates().count()

+--------------------+
|            location|
+--------------------+
|     Gainesville, FL|
|Atlantic City-Ham...|
|Deltona-Daytona B...|
|San Diego-Carlsba...|
|Cleveland-Elyria, OH|
+--------------------+
only showing top 5 rows



114

### Simplify "location"

In [9]:
# create column "state" with Abbreviation of US states extracted from "location" column
df_clean = df_clean.withColumn('state', trim(split(col('location'),',').getItem(1)))

In [10]:
# check unique "state" 
df_clean.select('state').dropDuplicates().count()

58

In [11]:
df_clean.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId',
 'state']

In [12]:
# drop columns
df_clean = df_clean.drop('location','auth','firstName','lastName','method','status')

### Create 'epoch time' from timestamp

In [13]:
# convert ts to epoch time and create month, date, hour columns out of it
df_clean = df_clean.withColumn('epoch_time', from_unixtime(col('ts').cast(LongType())/1000).cast(TimestampType()))
df_clean = df_clean.withColumn('date', from_unixtime(col('ts')/1000).cast(DateType()))
#df_clean = df_clean.withColumn('month', month(col('epoch_time')))
#df_clean = df_clean.withColumn('hour', hour(col('epoch_time')))  
#df_clean.show(5)

### Simplify "userAgent"

In [14]:
agent = udf(lambda x: str(re.findall(r'\((.*?)\)', x)[0].split(";")[0].split()[0]) if x is not None else None, StringType())

In [15]:
df_clean = df_clean.withColumn("agent", agent(df_clean.userAgent))

In [16]:
df_clean = df_clean.drop('userAgent')
#df_clean.show(5)

### Define Churn
Create a column "Churn" to use as the label for model. Using the "Cancellation Confirmation" events to define churn, which happen for both paid and free users. 

In [19]:
# find out users who churned their service
df_clean.filter("page = 'Cancellation Confirmation'").show(5,False)

+------+------+-------------+------+-----+-------------------------+-------------+---------+----+-------------+------+-----------+-------------------+----------+---------+
|artist|gender|itemInSession|length|level|page                     |registration |sessionId|song|ts           |userId|state      |epoch_time         |date      |agent    |
+------+------+-------------+------+-----+-------------------------+-------------+---------+----+-------------+------+-----------+-------------------+----------+---------+
|null  |M     |104          |null  |paid |Cancellation Confirmation|1535623466000|514      |null|1538943990000|18    |MO-KS      |2018-10-07 20:26:30|2018-10-07|Macintosh|
|null  |M     |56           |null  |paid |Cancellation Confirmation|1537167593000|540      |null|1539033046000|32    |AZ         |2018-10-08 21:10:46|2018-10-08|iPhone   |
|null  |M     |10           |null  |free |Cancellation Confirmation|1533157139000|174      |null|1539318918000|125   |TX         |2018-10-12

In [20]:
# list activities of a user who churned the service
df_clean.select(["userId","ts","page", "level", "song"]).where(df_clean.userId == "18").show() #on mini data
#df_clean.select(["userId", "ts", "auth","page", "level"]).where(df_clean.userId == "1768454").sort("ts").show() #on whole data

+------+-------------+--------+-----+--------------------+
|userId|           ts|    page|level|                song|
+------+-------------+--------+-----+--------------------+
|    18|1538499917000|    Home| paid|                null|
|    18|1538499933000|NextSong| paid|A Beggar On A Bea...|
|    18|1538500208000|NextSong| paid|...slowdance On T...|
|    18|1538500476000|NextSong| paid|       St. Apollonia|
|    18|1538500654000|NextSong| paid|      Drunk Stripper|
|    18|1538500842000|NextSong| paid|In League With Satan|
|    18|1538500856000|Settings| paid|                null|
|    18|1538501009000|NextSong| paid|           The Quest|
|    18|1538501340000|NextSong| paid|    Waking The Demon|
|    18|1538501587000|NextSong| paid|Why Do You Let Me...|
|    18|1538501740000|NextSong| paid|High (Album Version)|
|    18|1538501984000|NextSong| paid|Remember Me_ I'm ...|
|    18|1538502021000|Settings| paid|                null|
|    18|1538502124000|NextSong| paid|          Continent

In [17]:
# lable churn event,1 for churn, o for not
flag_churn_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
df_clean = df_clean.withColumn("Churn", flag_churn_event("page"))

# add churn user flag
df_clean = df_clean.withColumn("label", max('Churn').over(Window.partitionBy('UserId')))

In [18]:
# list activities of a user who churned the service
#df_clean.select(["userId", "ts","page","level","Churn","label"]).where(df_clean.userId == "1768454").show(5) #on whole data
df_clean.select(["userId", "ts","page","level","Churn","label"]).where(df_clean.userId == "18").show(5) # on mini data

+------+-------------+--------+-----+-----+-----+
|userId|           ts|    page|level|Churn|label|
+------+-------------+--------+-----+-----+-----+
|    18|1538499917000|    Home| paid|    0|    1|
|    18|1538499933000|NextSong| paid|    0|    1|
|    18|1538500208000|NextSong| paid|    0|    1|
|    18|1538500476000|NextSong| paid|    0|    1|
|    18|1538500654000|NextSong| paid|    0|    1|
+------+-------------+--------+-----+-----+-----+
only showing top 5 rows



In [19]:
df_clean.show(5)

+--------------------+---------+------+-------------+---------+-----+------+---------+-------------+---------+--------------------+------+-------------+------+-----+-------------------+----------+-----+------+-----+-----+
|              artist|     auth|gender|itemInSession|   length|level|method|     page| registration|sessionId|                song|status|           ts|userId|state|         epoch_time|      date|month| agent|Churn|label|
+--------------------+---------+------+-------------+---------+-----+------+---------+-------------+---------+--------------------+------+-------------+------+-----+-------------------+----------+-----+------+-----+-----+
|Sleeping With Sirens|Logged In|     F|            0|202.97098| free|   PUT| NextSong|1538016340000|       31|Captain Tyin Knot...|   200|1539003534000|100010|   CT|2018-10-08 12:58:54|2018-10-08|   10|iPhone|    0|    0|
|Francesca Battist...|Logged In|     F|            1|196.54485| free|   PUT| NextSong|1538016340000|       31|Be

### Define downgrade 

Create a column "downgraded" using the "Downgrade" events. Flag users who downgraded at least once as 1 and users who never downgraded as 0. 

In [20]:
# find out users who downgraded their service
df_clean.filter("page = 'Downgrade'").show(1)

+------+---------+------+-------------+------+-----+------+---------+-------------+---------+----+------+-------------+------+--------+-------------------+----------+-----+------+-----+-----+
|artist|     auth|gender|itemInSession|length|level|method|     page| registration|sessionId|song|status|           ts|userId|   state|         epoch_time|      date|month| agent|Churn|label|
+------+---------+------+-------------+------+-----+------+---------+-------------+---------+----+------+-------------+------+--------+-------------------+----------+-----+------+-----+-----+
|  null|Logged In|     M|           55|  null| paid|   GET|Downgrade|1536269906000|      163|null|   200|1539857842000|200002|IL-IN-WI|2018-10-18 10:17:22|2018-10-18|   10|iPhone|    0|    0|
+------+---------+------+-------------+------+-----+------+---------+-------------+---------+----+------+-------------+------+--------+-------------------+----------+-----+------+-----+-----+
only showing top 1 row



In [21]:
# list activities of a user who downgraded the service
#df_clean.select(["userId", "ts", "auth","page", "level"]).where(df_clean.userId == "1001393").sort("ts").show() # on whole data
df_clean.select(["userId", "ts", "auth","page", "level"]).where(df_clean.userId == "200002").sort("ts").show() # on mini data

+------+-------------+---------+---------------+-----+
|userId|           ts|     auth|           page|level|
+------+-------------+---------+---------------+-----+
|200002|1538393430000|Logged In|       NextSong| free|
|200002|1538393476000|Logged In|    Roll Advert| free|
|200002|1538393635000|Logged In|       NextSong| free|
|200002|1538393806000|Logged In|       NextSong| free|
|200002|1538393807000|Logged In|      Thumbs Up| free|
|200002|1538393977000|Logged In|       NextSong| free|
|200002|1538394156000|Logged In|Add to Playlist| free|
|200002|1538501988000|Logged In|       NextSong| free|
|200002|1538502256000|Logged In|       NextSong| free|
|200002|1538502641000|Logged In|       NextSong| free|
|200002|1538502773000|Logged In|       NextSong| free|
|200002|1538502918000|Logged In|    Roll Advert| free|
|200002|1538502945000|Logged In|       NextSong| free|
|200002|1538503168000|Logged In|       NextSong| free|
|200002|1538503241000|Logged In|          About| free|
|200002|15

In [18]:
# lable downgrade event,1 for downgraded, 0 for not
flag_downgrade_event = udf(lambda x: 1 if x == "Downgrade" else 0, IntegerType())
df_clean = df_clean.withColumn("Downgrade", flag_downgrade_event("page"))

# add downgraded user flag
df_clean = df_clean.withColumn("downgraded", max('Downgrade').over(Window.partitionBy('UserId')))

In [23]:
# list activities of a user who downgraded the service
#df_clean.select(["userId", "auth","page", "level", "downgraded","label"]).where(df_clean.userId == "1001393").sort("ts").show() # on whole data
df_clean.select(["userId", "auth","page", "level", "downgraded","label"]).where(df_clean.userId == "200002").sort("ts").show() # on whole data


+------+---------+---------------+-----+----------+-----+
|userId|     auth|           page|level|downgraded|label|
+------+---------+---------------+-----+----------+-----+
|200002|Logged In|       NextSong| free|         1|    0|
|200002|Logged In|    Roll Advert| free|         1|    0|
|200002|Logged In|       NextSong| free|         1|    0|
|200002|Logged In|       NextSong| free|         1|    0|
|200002|Logged In|      Thumbs Up| free|         1|    0|
|200002|Logged In|       NextSong| free|         1|    0|
|200002|Logged In|Add to Playlist| free|         1|    0|
|200002|Logged In|       NextSong| free|         1|    0|
|200002|Logged In|       NextSong| free|         1|    0|
|200002|Logged In|       NextSong| free|         1|    0|
|200002|Logged In|       NextSong| free|         1|    0|
|200002|Logged In|    Roll Advert| free|         1|    0|
|200002|Logged In|       NextSong| free|         1|    0|
|200002|Logged In|       NextSong| free|         1|    0|
|200002|Logged

In [19]:
# delete 'Downgrade','Churn' columns
df_clean = df_clean.drop('Downgrade','Churn')

In [19]:
df_clean.columns

['artist',
 'gender',
 'itemInSession',
 'length',
 'level',
 'page',
 'registration',
 'sessionId',
 'song',
 'ts',
 'userId',
 'state',
 'epoch_time',
 'date',
 'agent',
 'label',
 'downgraded']

In [20]:
df_clean.count()

278154

In [20]:
# subset data
df_page = df_clean.select('label','userId','page')

In [22]:
# create smaller data
# delete missing values in "artist" column
df_clean = df_clean.filter(df_clean.artist != "")

### Get count of missing values
df_clean.select([count(when(col(c).isNull(), c)).alias(c) for c in df_clean.columns]).show() 

# there is no missing value in any column after deleting missing "artist"
df_clean.count()

+------+------+-------------+------+-----+----+------------+---------+----+---+------+-----+----------+----+-----+-----+----------+
|artist|gender|itemInSession|length|level|page|registration|sessionId|song| ts|userId|state|epoch_time|date|agent|label|downgraded|
+------+------+-------------+------+-----+----+------------+---------+----+---+------+-----+----------+----+-----+-----+----------+
|     0|     0|            0|     0|    0|   0|           0|        0|   0|  0|     0|    0|         0|   0|    0|    0|         0|
+------+------+-------------+------+-----+----+------------+---------+----+---+------+-----+----------+----+-----+-----+----------+



228108

In [23]:
# check unique userId 
df_clean.select('userId').dropDuplicates().count() # unique userId number did not change

225

## Explore Data (plotting codes are separated to another file because of memory overuse problem)
Perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. Exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

## Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

### Registered days


In [21]:
cancellation_df = df_clean.select('userId','ts').groupBy('userId').agg(max('ts').alias('lastinteraction'))
df_clean = cancellation_df.join(df_clean, on='userId').withColumn('registered_days', ((col('lastinteraction')-col('registration'))/86400000).cast(IntegerType()))
df_clean = df_clean.drop('lastinteraction','registration')
#df_clean.show(2)
del cancellation_df

### Latest "level" of users before they churn

In [22]:
level_df = df_clean.select('ts','userId','level').orderBy('ts', ascending=False).groupBy('userId').agg(first('level').alias('valid_level'))    
df_clean = df_clean.drop('level')
df_clean = df_clean.join(level_df, on='userId')
#df_clean.show(2)
del level_df

### Number of songs per day

In [23]:
temp_daily_song = df_clean.select('userId','date','song').groupBy('userId','date').agg(countDistinct('song').alias('songs')).sort('userId')
daily_song = temp_daily_song.groupBy('userId').avg('songs').withColumnRenamed('avg(songs)', 'avg_daily_song')
df_clean = df_clean.join(daily_song, on='userId')
del temp_daily_song,daily_song
df_clean.show(2)

+------+--------------------+------+-------------+---------+--------+---------+--------------------+-------------+-----+-------------------+----------+------+-----+----------+---------------+-----------+------------------+
|userId|              artist|gender|itemInSession|   length|    page|sessionId|                song|           ts|state|         epoch_time|      date| agent|label|downgraded|registered_days|valid_level|    avg_daily_song|
+------+--------------------+------+-------------+---------+--------+---------+--------------------+-------------+-----+-------------------+----------+------+-----+----------+---------------+-----------+------------------+
|100010|Sleeping With Sirens|     F|            0|202.97098|NextSong|       31|Captain Tyin Knot...|1539003534000|   CT|2018-10-08 12:58:54|2018-10-08|iPhone|    0|         0|             55|       free|39.142857142857146|
|100010|Francesca Battist...|     F|            1|196.54485|NextSong|       31|Beautiful_ Beauti...|15390037

### Number of songs per session

In [24]:
# create new feature
song_per_session_df = df_clean.select('page','label', 'userId', 'sessionId').where('page == "NextSong"').groupby(['label', 'userId', 'sessionId']).count()\
    .groupby(['label', 'userId']).agg({'count': 'avg'})\
    .withColumnRenamed('avg(count)', 'songs_per_session')

In [25]:
# join new feature to the dataframe
df_clean = df_clean.join(song_per_session_df.drop('label'), on='userId')
#df_clean.show(2)
del song_per_session_df

### Average session duration

In [26]:
# session duration for each user
session_duration = df_clean.select('userId','sessionId','ts').groupBy('userId','sessionId').agg(((max('ts')-min('ts'))/1000/3600).alias('activesession'))
# average session duration for each user
session_duration_df = session_duration.groupBy('userId').avg('activesession').withColumnRenamed('avg(activesession)', 'avg_session')

# join dataframe to create new column
df_clean = df_clean.join(session_duration_df, on='userId')
#df_clean.show(2)
del session_duration,session_duration_df

### Friends added

In [27]:
# count total number of "add friend"
friends_df = df_page.where('page == "Add Friend"').groupby(['label', 'userId']).count()\
    .groupby(['label', 'userId']).agg({'count': 'avg'})\
    .withColumnRenamed('avg(count)', 'friends')
#friends_df.show(2)

In [28]:
# join dataframe to create new column "friends"
df_clean = df_clean.join(friends_df.drop('label'), on='userId',how='left')
#df_clean.show(2)
del friends_df

### Number of "thumbs up"

In [29]:
# count total number of "thumbs up"
thumbs_up_df = df_page.where('page == "Thumbs Up"').groupby(['label','userId']).count()\
    .groupby(['label','userId']).agg({'count': 'avg'})\
    .withColumnRenamed('avg(count)','thumbs_ups')

In [30]:
# join dataframe to create new column "thumbs_ups"
df_clean = df_clean.join(thumbs_up_df.drop('label'), on='userId',how='left')
#df_clean.show(2)
del thumbs_up_df

### Number of "thumbs down"

In [31]:
# count total number of "thumbs down"
thumbs_down_df = df_page.where('page == "Thumbs Down"').groupby(['label','userId']).count()\
    .groupby(['label','userId']).agg({'count': 'avg'})\
    .withColumnRenamed('avg(count)','thumbs_downs')

In [32]:
# join dataframe to create new column "thumbs_down"
df_clean = df_clean.join(thumbs_down_df.drop('label'), on='userId',how='left')
#df_clean.show(2)
del thumbs_down_df

### Times "add to playlist"

In [33]:
# count number of "add to playlist"
add_playlist_df = df_page.where('page == "Add to Playlist"').groupby(['label', 'userId']).count()\
    .groupby(['label', 'userId']).agg({'count': 'avg'})\
    .withColumnRenamed('avg(count)', 'add_playlist')

In [34]:
# join dataframe to create new column "add_playlist"
df_clean = df_clean.join(add_playlist_df.drop('label'), on='userId',how='left')
#df_clean.show(2)
del add_playlist_df

### Times "Roll Advert"

In [35]:
# count number of "roll advert"
roll_advert_df = df_page.where('page == "Add to Playlist"').groupby(['label', 'userId']).count()\
    .groupby(['label', 'userId']).agg({'count': 'avg'})\
    .withColumnRenamed('avg(count)', 'roll_advert')

In [36]:
# join dataframe to create new column "roll_advert"
df_clean = df_clean.join(roll_advert_df.drop('label'), on='userId',how='left')
#df_clean.show(2)
del roll_advert_df

In [37]:
df_clean.columns

['userId',
 'artist',
 'gender',
 'itemInSession',
 'length',
 'page',
 'sessionId',
 'song',
 'ts',
 'state',
 'epoch_time',
 'date',
 'agent',
 'label',
 'downgraded',
 'registered_days',
 'valid_level',
 'avg_daily_song',
 'songs_per_session',
 'avg_session',
 'friends',
 'thumbs_ups',
 'thumbs_downs',
 'add_playlist',
 'roll_advert']

In [40]:
# save data for visualization
df_clean.select('userId','label','downgraded','gender','valid_level','agent','state',\
          'registered_days','avg_daily_song','songs_per_session','avg_session',\
          'friends','thumbs_ups','thumbs_downs','add_playlist','roll_advert').dropDuplicates(['userId']).write.save("df_visual", format="json")

In [41]:
# assemble features for modeling
df_model = df_clean.select(['userId','label','downgraded','gender','valid_level','agent',\
                            'registered_days','avg_daily_song','songs_per_session','avg_session',\
                            'friends','thumbs_ups','thumbs_downs','add_playlist','roll_advert']).dropDuplicates(['userId'])

In [42]:
df_model.count()

225

In [44]:
### Get count of missing values
df_model.select([count(when(col(c).isNull(), c)).alias(c) for c in df_model.columns]).show() 

+------+-----+----------+------+-----------+-----+---------------+--------------+-----------------+-----------+-------+----------+------------+------------+-----------+
|userId|label|downgraded|gender|valid_level|agent|registered_days|avg_daily_song|songs_per_session|avg_session|friends|thumbs_ups|thumbs_downs|add_playlist|roll_advert|
+------+-----+----------+------+-----------+-----+---------------+--------------+-----------------+-----------+-------+----------+------------+------------+-----------+
|     0|    0|         0|     0|          0|    0|              0|             0|                0|          0|     19|         5|          22|          10|         10|
+------+-----+----------+------+-----------+-----+---------------+--------------+-----------------+-----------+-------+----------+------------+------------+-----------+



In [43]:
# format decimal
df_model = df_model.withColumn("daily_song", bround(df_model.avg_daily_song,1))
df_model = df_model.withColumn("session_song", bround(df_model.songs_per_session,1))
df_model = df_model.withColumn("session_duration", bround(df_model.avg_session,1))

# replace missing value with 0
df_model = df_model.na.fill(0)
df_model.columns


['userId',
 'label',
 'downgraded',
 'gender',
 'valid_level',
 'agent',
 'registered_days',
 'avg_daily_song',
 'songs_per_session',
 'avg_session',
 'friends',
 'thumbs_ups',
 'thumbs_downs',
 'add_playlist',
 'roll_advert',
 'daily_song',
 'session_song',
 'session_duration']

In [45]:
# drop old columns
df_model = df_model.drop('avg_daily_song','songs_per_session','avg_session','userId')

In [46]:
df_model.show(5)

+-----+----------+------+-----------+---------+---------------+-------+----------+------------+------------+-----------+----------+------------+----------------+
|label|downgraded|gender|valid_level|    agent|registered_days|friends|thumbs_ups|thumbs_downs|add_playlist|roll_advert|daily_song|session_song|session_duration|
+-----+----------+------+-----------+---------+---------------+-------+----------+------------+------------+-----------+----------+------------+----------------+
|    0|         0|     F|       free|   iPhone|             55|    4.0|      17.0|         5.0|         7.0|        7.0|      39.1|        39.3|             2.6|
|    0|         1|     M|       paid|   iPhone|             70|    4.0|      21.0|         6.0|         8.0|        8.0|      55.0|        64.5|             4.4|
|    1|         0|     M|       free|Macintosh|             71|    0.0|       0.0|         0.0|         0.0|        0.0|       8.0|         8.0|             0.5|
|    0|         1|     F|   

In [47]:
# check columns
df_model.printSchema()

root
 |-- label: integer (nullable = true)
 |-- downgraded: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- valid_level: string (nullable = true)
 |-- agent: string (nullable = true)
 |-- registered_days: integer (nullable = true)
 |-- friends: double (nullable = false)
 |-- thumbs_ups: double (nullable = false)
 |-- thumbs_downs: double (nullable = false)
 |-- add_playlist: double (nullable = false)
 |-- roll_advert: double (nullable = false)
 |-- daily_song: double (nullable = false)
 |-- session_song: double (nullable = false)
 |-- session_duration: double (nullable = false)



In [48]:
### Get count of missing values
df_model.select([count(when(col(c).isNull(), c)).alias(c) for c in df_model.columns]).show() 

+-----+----------+------+-----------+-----+---------------+-------+----------+------------+------------+-----------+----------+------------+----------------+
|label|downgraded|gender|valid_level|agent|registered_days|friends|thumbs_ups|thumbs_downs|add_playlist|roll_advert|daily_song|session_song|session_duration|
+-----+----------+------+-----------+-----+---------------+-------+----------+------------+------------+-----------+----------+------------+----------------+
|    0|         0|     0|          0|    0|              0|      0|         0|           0|           0|          0|         0|           0|               0|
+-----+----------+------+-----------+-----+---------------+-------+----------+------------+------------+-----------+----------+------------+----------------+



In [49]:
# save data for modeling
df_model.write.save("df_model", format="json")

### Transform categorical columns with StringIndexer

In [50]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df_model) for column in ['gender','valid_level','agent'] ]
pipeline = Pipeline(stages=indexers)
df_r = pipeline.fit(df_model).transform(df_model)
df_model = df_r.drop('gender','valid_level','agent')

In [49]:
df_model.show(5)

+-----+----------+---------------+-------+----------+------------+------------+-----------+----------+------------+----------------+------------+-----------------+-----------+
|label|downgraded|registered_days|friends|thumbs_ups|thumbs_downs|add_playlist|roll_advert|daily_song|session_song|session_duration|gender_index|valid_level_index|agent_index|
+-----+----------+---------------+-------+----------+------------+------------+-----------+----------+------------+----------------+------------+-----------------+-----------+
|    0|         0|             55|    4.0|      17.0|         5.0|         7.0|        7.0|      39.1|        39.3|             2.6|         1.0|              1.0|        2.0|
|    0|         1|             70|    4.0|      21.0|         6.0|         8.0|        8.0|      55.0|        64.5|             4.4|         0.0|              0.0|        2.0|
|    1|         0|             71|    0.0|       0.0|         0.0|         0.0|        0.0|       8.0|         8.0|     

### Standard sclarizer
Use StandardScaler to scalerize the created “scaled_feature” column

In [50]:
df_model.columns

['label',
 'downgraded',
 'registered_days',
 'friends',
 'thumbs_ups',
 'thumbs_downs',
 'add_playlist',
 'roll_advert',
 'daily_song',
 'session_song',
 'session_duration',
 'gender_index',
 'valid_level_index',
 'agent_index']

In [51]:
# assembeling numeric features to create a vector
cols=['registered_days','friends','thumbs_ups','thumbs_downs','add_playlist','roll_advert',\
      'daily_song','session_song','session_duration']

assembler = VectorAssembler(inputCols=cols,outputCol="features")

# use the transform method to transform df
df_model = assembler.transform(df_model)
df_model.select("features").show(truncate=False)

+---------------------------------------------------+
|features                                           |
+---------------------------------------------------+
|[55.0,4.0,17.0,5.0,7.0,7.0,39.1,39.3,2.6]          |
|[70.0,4.0,21.0,6.0,8.0,8.0,55.0,64.5,4.4]          |
|(9,[0,6,7,8],[71.0,8.0,8.0,0.5])                   |
|[131.0,74.0,171.0,41.0,118.0,118.0,121.1,145.7,9.6]|
|[19.0,28.0,100.0,21.0,52.0,52.0,158.5,211.1,14.5]  |
|[72.0,1.0,7.0,1.0,5.0,5.0,18.8,21.4,1.5]           |
|[56.0,31.0,81.0,14.0,59.0,59.0,99.8,136.7,8.8]     |
|[110.0,33.0,163.0,29.0,72.0,72.0,90.5,81.2,5.4]    |
|[23.0,11.0,58.0,3.0,24.0,24.0,100.8,136.7,9.1]     |
|[85.0,6.0,17.0,3.0,7.0,7.0,42.7,42.8,3.1]          |
|[66.0,41.0,96.0,17.0,38.0,38.0,90.4,120.5,8.3]     |
|[23.0,3.0,11.0,0.0,1.0,1.0,27.7,28.0,1.8]          |
|[53.0,29.0,86.0,16.0,61.0,61.0,135.8,179.7,13.6]   |
|[124.0,6.0,40.0,9.0,20.0,20.0,40.1,40.4,2.7]       |
|[66.0,41.0,95.0,24.0,67.0,67.0,118.2,138.0,9.4]    |
|[74.0,63.0,303.0,28.0,113.0

In [52]:
# standardize numeric feature vector
standardscaler=StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
df_model = standardscaler.fit(df_model).transform(df_model)
df_model.select("Scaled_features").show(2,False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Scaled_features                                                                                                                                                                |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[1.46095433566683,0.19434724774254747,0.2596294873653501,0.38233662155526527,0.21395831049077527,0.21395831049077527,1.2518757615061304,0.922113675073389,0.8826912851795525]  |
|[1.859396427212329,0.19434724774254747,0.32071877851013836,0.4588039458663183,0.24452378341802888,0.24452378341802888,1.7609505596633548,1.5133926728303715,1.4937852518423198]|
+-------------------------------------------------------------------------------------------------------------

In [53]:
df_model.select("Scaled_features").show(3,False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Scaled_features                                                                                                                                                                |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[1.461623603743267,0.19434724774254747,0.2596294873653501,0.38233662155526527,0.21395831049077527,0.21395831049077527,1.2465675015038198,0.922113675073389,0.8762953534378627] |
|[1.8602482229459762,0.19434724774254747,0.32071877851013836,0.4588039458663183,0.24452378341802888,0.24452378341802888,1.7534836977675214,1.5133926728303715,1.482961367356383]|
|(9,[0,6,7,8],[1.8868231975594902,0.2550521742207304,0.18770761833554994,0.16851833719958897])                

### Combine all features in one single feature vector

In [54]:
df_model.columns

['label',
 'downgraded',
 'registered_days',
 'friends',
 'thumbs_ups',
 'thumbs_downs',
 'add_playlist',
 'roll_advert',
 'daily_song',
 'session_song',
 'session_duration',
 'gender_index',
 'valid_level_index',
 'agent_index',
 'features',
 'Scaled_features']

In [53]:
cols=['Scaled_features','downgraded','gender_index','valid_level_index','agent_index']
assembler = VectorAssembler(inputCols=cols,outputCol='exp_features')
# use the transform method to transform df
df_model = assembler.transform(df_model)

In [54]:
df_model.select('exp_features').show(10,truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|exp_features                                                                                                                                                                                     |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[1.46095433566683,0.19434724774254747,0.2596294873653501,0.38233662155526527,0.21395831049077527,0.21395831049077527,1.2518757615061304,0.922113675073389,0.8826912851795525,0.0,1.0,1.0,2.0]    |
|[1.859396427212329,0.19434724774254747,0.32071877851013836,0.4588039458663183,0.24452378341802888,0.24452378341802888,1.7609505596633548,1.5133926728303715,1.4937852518423198,1.0,0.0,0.0,2.0]  |
|(13,[0,6,7,8,11,12]

In [55]:
# save data for modeling
df_model.write.save("df_model_vector", format="json")

## Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

### Train test split
As a first step break your data set into 80% of training data and set aside 20%. Set random seed to `42`.

In [56]:
rest, validation = df_model.randomSplit([0.8, 0.2], seed=42)
print("We have %d training examples and %d test examples." % (rest.count(), validation.count())

### Check imbalance in the dataset

In [58]:
dataset_size = float(rest.select('label').count())
numPositives = rest.select('label').where('label == 1').count()
per_ones = (float(numPositives)/float(dataset_size))*100
numNegatives = float(dataset_size-numPositives)
print('The number of ones are {}'.format(numPositives))
print('Percentage of ones are {}'.format(per_ones))

The number of ones are 42
Percentage of ones are 21.98952879581152


### Class weighing

In [59]:
BalancingRatio= numNegatives/dataset_size
print('BalancingRatio = {}'.format(BalancingRatio))

BalancingRatio = 0.7801047120418848


In [60]:
# creating a new column named “classWeights” in the “rest” dataset
rest = rest.withColumn("classWeights", when(rest.label == 1,BalancingRatio).otherwise(1-BalancingRatio))
rest.select("classWeights").show(5)

+------------------+
|      classWeights|
+------------------+
|0.2198952879581152|
|0.7801047120418848|
|0.2198952879581152|
|0.7801047120418848|
|0.7801047120418848|
+------------------+
only showing top 5 rows



### Logistic regression model

In [61]:
lr = LogisticRegression(labelCol="label", featuresCol="exp_features",weightCol="classWeights",maxIter=10)
model = lr.fit(rest)
predict_rest = model.transform(rest)
predict_val = model.transform(validation)
predict_val.select("label","prediction").show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|    0|       1.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 10 rows



In [62]:
# Evaluating the LR model using BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="label")
predict_val.select("label","rawPrediction","prediction","probability").show(5)
#print("The area under ROC for train set is {}".format(evaluator.evaluate(predict_rest)))
#print("The area under ROC for test set is {}".format(evaluator.evaluate(predict_val)))

+-----+--------------------+----------+--------------------+
|label|       rawPrediction|prediction|         probability|
+-----+--------------------+----------+--------------------+
|    0|[-0.6043482042729...|       1.0|[0.35334952437924...|
|    0|[4.94462726323463...|       0.0|[0.99292878947127...|
|    0|[1.20590043290124...|       0.0|[0.76957277421950...|
|    1|[-0.3319441140787...|       1.0|[0.41776766558045...|
|    0|[0.63986233410138...|       0.0|[0.65472234037888...|
+-----+--------------------+----------+--------------------+
only showing top 5 rows



In [63]:
#F1 score
f1_score_evaluator = MulticlassClassificationEvaluator(metricName='f1')
f1_score_rest = f1_score_evaluator.evaluate(predict_rest.select(col('label'), col('prediction')))
f1_score_val = f1_score_evaluator.evaluate(predict_val.select(col('label'), col('prediction')))
print('The F1 score on the train set is {:.2%}'.format(f1_score_rest))
print('The F1 score on the test set is {:.2%}'.format(f1_score_val)) 

#area under ROC 
auc_evaluator = BinaryClassificationEvaluator()
roc_value_rest = auc_evaluator.evaluate(predict_rest, {auc_evaluator.metricName: "areaUnderROC"})
roc_value_val = auc_evaluator.evaluate(predict_val, {auc_evaluator.metricName: "areaUnderROC"})
print('The areaUnderROC on the train set is {:.2%}'.format(roc_value_rest))
print('The areaUnderROC on the test set is {:.2%}'.format(roc_value_val))

The F1 score on the train set is 78.51%
The F1 score on the test set is 75.63%
The areaUnderROC on the train set is 82.89%
The areaUnderROC on the test set is 83.75%


### Random forest model

In [64]:
rf = RandomForestClassifier(labelCol="label", featuresCol="exp_features")#,maxIter=10)#weightCol="classWeights") 
model = rf.fit(rest)
predict_rest = model.transform(rest)
predict_val = model.transform(validation)
predict_val.select("label","prediction").show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       1.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 10 rows



In [65]:
# Evaluating the RF model using BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="label")
predict_val.select("label","rawPrediction","prediction","probability").show(5)

#F1 score
f1_score_evaluator = MulticlassClassificationEvaluator(metricName='f1')
f1_score_rest = f1_score_evaluator.evaluate(predict_rest.select(col('label'), col('prediction')))
f1_score_val = f1_score_evaluator.evaluate(predict_val.select(col('label'), col('prediction')))
print('The F1 score on the train set is {:.2%}'.format(f1_score_rest))
print('The F1 score on the test set is {:.2%}'.format(f1_score_val)) 

#area under ROC 
auc_evaluator = BinaryClassificationEvaluator()
roc_value_rest = auc_evaluator.evaluate(predict_rest, {auc_evaluator.metricName: "areaUnderROC"})
roc_value_val = auc_evaluator.evaluate(predict_val, {auc_evaluator.metricName: "areaUnderROC"})
print('The areaUnderROC on the train set is {:.2%}'.format(roc_value_rest))
print('The areaUnderROC on the test set is {:.2%}'.format(roc_value_val))

+-----+--------------------+----------+--------------------+
|label|       rawPrediction|prediction|         probability|
+-----+--------------------+----------+--------------------+
|    0|[14.9518064118071...|       0.0|[0.74759032059035...|
|    0|[18.6393729079922...|       0.0|[0.93196864539961...|
|    0|[8.57377163122777...|       1.0|[0.42868858156138...|
|    1|[18.3776037455221...|       0.0|[0.91888018727610...|
|    0|[17.7137869358082...|       0.0|[0.88568934679041...|
+-----+--------------------+----------+--------------------+
only showing top 5 rows

The F1 score on the train set is 90.85%
The F1 score on the test set is 72.74%
The areaUnderROC on the train set is 98.51%
The areaUnderROC on the test set is 66.25%


### Gradient boosted trees model

In [57]:
# Initialize Gradient-Boosted Tree object
gbt = GBTClassifier(labelCol="label", featuresCol="exp_features", maxIter=10)

#Fit the model to the data
model = gbt.fit(rest)

# Score the training and testing dataset using fitted model for evaluation purposes
predict_rest = model.transform(rest)
predict_val = model.transform(validation)
predict_val.select("label","prediction").show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       1.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
+-----+----------+
only showing top 10 rows



In [58]:
# Evaluating the gbt model using BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="label")
predict_val.select("label","rawPrediction","prediction","probability").show(5)

#F1 score
f1_score_evaluator = MulticlassClassificationEvaluator(metricName='f1')
f1_score_rest = f1_score_evaluator.evaluate(predict_rest.select(col('label'), col('prediction')))
f1_score_val = f1_score_evaluator.evaluate(predict_val.select(col('label'), col('prediction')))
print('The F1 score on the train set is {:.2%}'.format(f1_score_rest))
print('The F1 score on the test set is {:.2%}'.format(f1_score_val)) 

#area under ROC 
auc_evaluator = BinaryClassificationEvaluator()
roc_value_rest = auc_evaluator.evaluate(predict_rest, {auc_evaluator.metricName: "areaUnderROC"})
roc_value_val = auc_evaluator.evaluate(predict_val, {auc_evaluator.metricName: "areaUnderROC"})
print('The areaUnderROC on the train set is {:.2%}'.format(roc_value_rest))
print('The areaUnderROC on the test set is {:.2%}'.format(roc_value_val))

+-----+--------------------+----------+--------------------+
|label|       rawPrediction|prediction|         probability|
+-----+--------------------+----------+--------------------+
|    0|[1.24708207708245...|       0.0|[0.92373169149972...|
|    0|[1.23999313964552...|       0.0|[0.92272681955453...|
|    0|[-0.7454301479447...|       1.0|[0.18379263682647...|
|    1|[-0.0790917318979...|       1.0|[0.46053638798985...|
|    0|[0.87549905665144...|       0.0|[0.85207864900675...|
+-----+--------------------+----------+--------------------+
only showing top 5 rows

The F1 score on the train set is 98.42%
The F1 score on the test set is 81.73%
The areaUnderROC on the train set is 99.92%
The areaUnderROC on the test set is 77.08%


### Hyperparameter tuning
Create a parameter grid for tuning the model and perform cross validation

In [68]:
# for random forest model
rfparamGrid = ParamGridBuilder()\
    .addGrid(rf.maxDepth,[3,5,10])\
    .addGrid(rf.numTrees, [20,50,75])\
    .addGrid(rf.impurity,["entropy", "gini"])\
    .build()

In [ ]:
# Create 3-fold CrossValidator
cv = CrossValidator(estimator=rf, estimatorParamMaps=rfparamGrid, evaluator=evaluator, numFolds=3)

# Run cross validations
cvModel = cv.fit(rest)

# this will likely take a fair amount of time because of the amount of models that we're creating and testing
predict_rest = cvModel.transform(rest)
predict_validation = cvModel.transform(validation)

print("The area under ROC for train set after CV  is {}".format(evaluator.evaluate(predict_rest)))
print("The area under ROC for test set after CV  is {}".format(evaluator.evaluate(predict_validation)))

In [63]:
# for Gradient-Boosted Trees model
gbparamGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [2, 5, 10])
             .addGrid(gbt.maxBins, [10, 20, 40])
             .addGrid(gbt.maxIter, [5, 10, 20])
             .build())

In [64]:
# Create 3-fold CrossValidator
cv = CrossValidator(estimator=gbt, estimatorParamMaps=gbparamGrid, evaluator=evaluator, numFolds=3)

# Run cross validations
cvModel = cv.fit(rest)

# this will likely take a fair amount of time because of the amount of models that we're creating and testing
predict_rest = cvModel.transform(rest)
predict_validation = cvModel.transform(validation)

print("The area under ROC for train set after CV  is {}".format(evaluator.evaluate(predict_rest)))
print("The area under ROC for test set after CV  is {}".format(evaluator.evaluate(predict_validation)))

The area under ROC for train set after CV  is 0.8100031959092361
The area under ROC for test set after CV  is 0.65625


In [65]:
#F1 score
f1_score_evaluator = MulticlassClassificationEvaluator(metricName='f1')
f1_score_rest = f1_score_evaluator.evaluate(predict_rest.select(col('label'), col('prediction')))
f1_score_val = f1_score_evaluator.evaluate(predict_val.select(col('label'), col('prediction')))
print('The F1 score after CV on the train set is {:.2%}'.format(f1_score_rest))
print('The F1 score after CV on the test set is {:.2%}'.format(f1_score_val))

The F1 score on the train set is 84.22%
The F1 score on the test set is 81.73%


## Future work: Testing selective sampling method
Because the training data set is imbalanced with more 0 labeled rows than 1, here I wanted to try if randomly selecting the same number of 0 rows as 1 rows might improve the model performance. 

In [ ]:
#number of stayed and churned users
df_model.groupBy('label').agg(countDistinct('userId').alias('user_count')).show()

In [ ]:
# random selection of same number of 0 labeled rows as the 1 labeled rows
df_model.where(df_model.label == "0").orderBy(rand()).limit(57)